# End of week 1 exercise

To demonstrate your familiarity with OpenAI API, and also Ollama, build a tool that takes a technical question,  
and responds with an explanation. This is a tool that you will be able to use yourself during the course!

In [7]:
# imports
import os
from dotenv import load_dotenv
from openai import OpenAI
from IPython.display import Markdown, display, update_display
import ollama

In [8]:
# constants

MODEL_GPT = 'gpt-4o-mini'
MODEL_LLAMA = 'llama3.2:1b'

In [9]:
# set up environment
load_dotenv(override=True)
api_key = os.getenv('OPENAI_API_KEY')

if api_key and api_key.startswith('sk-proj-') and len(api_key) > 10:
    print("API key looks good so far")
else:
    print("There might be a problem with your API key?")

# Create OpenAI client for GPT
openai_gpt = OpenAI()

# Note: Using native ollama library (faster than OpenAI-compatible API wrapper)
# No need to create a separate client - just use ollama.chat() directly

API key looks good so far


In [10]:
# here is the question; type over this to ask something new

question = """
Please explain what this code does and why:
yield from {book.get("author") for book in books if book.get("author")}
"""

In [11]:
# Get gpt-4o-mini to answer, with streaming
stream = openai_gpt.chat.completions.create(
    model=MODEL_GPT,
    messages=[
        {"role": "user", "content": question}
    ],
    stream=True
)

response = ""
display_handle = display(Markdown(""), display_id=True)
for chunk in stream:
    response += chunk.choices[0].delta.content or ''
    update_display(Markdown(response), display_id=display_handle.display_id)

The code snippet you provided uses Python's `yield from` syntax in conjunction with a set comprehension to yield values from a collection. Let's break it down step by step:

1. **Set Comprehension**: 
   ```python
   {book.get("author") for book in books if book.get("author")}
   ```
   This part of the code creates a set of authors extracted from a collection called `books`. Here's how it works:
   - `for book in books`: Iterates over each `book` in the `books` collection (which is expected to be a list or iterable of dictionaries).
   - `book.get("author")`: Retrieves the value associated with the key `"author"` from each `book` dictionary, if it exists.
   - `if book.get("author")`: This condition filters out any `None` or falsy values, meaning only books that have a valid author will contribute to the set.
   - The overall result is a set containing unique author names, as sets automatically handle duplicates.

2. **`yield from`**:
   ```python
   yield from ...
   ```
   The `yield from` expression is used to yield all values from an iterable. In this case, it yields all authors from the set created by the set comprehension. By using `yield from`, if the surrounding function is a generator, it allows the generator to provide each author one at a time, yielding control back to the caller with each author.

### Summary of What the Code Does:
- It iterates through a collection of books (`books`).
- It collects the authors of those books into a set to ensure uniqueness while filtering out any empty or missing author entries.
- It yields each author name one at a time to whoever is calling the generator function.

### Why Use This Code:
- **Efficiency**: Using a set comprehension ensures that you only have unique authors without needing additional logic to filter out duplicates.
- **Generator Patterns**: By using `yield from`, it allows for lazy evaluation of authors, meaning that authors are emitted one by one as needed, rather than all at once. This is particularly useful when dealing with large datasets where you may not want to load everything into memory simultaneously. 

In conclusion, this code snippet efficiently gathers and yields unique authors from a list of books, cleaning out any invalid entries in the process.

In [12]:
# Get Llama 3.2:1b to answer (using native ollama library - faster!)
response_llama = ollama.chat(
    model=MODEL_LLAMA,
    messages=[
        {"role": "user", "content": question}
    ]
)

result_llama = response_llama['message']['content']
display(Markdown(result_llama))

This code is using the `yield` keyword in Python, which allows it to generate a sequence of values over time. It's a part of the `__iter__` method of an object.

Here's what this specific line does:

```python
yield from {book.get("author") for book in books if book.get("author")}
```

1. `{book.get("author")`: This is creating an expression that yields (or returns) the value of `book.get("author")` for each iteration.
2. `for book in books`: This starts a loop over the list or other iterable (`books`) and assigns it to the variable `book`.
3. `if book.get("author")`: It filters this output to only include values where `get("author")` returns something. This is necessary because if you just iterate over all items without filtering, you'll get an error when trying to access `book.get("author")` (because the list might be empty).
4. `yield from ...`: Here's the important part: this expression creates a generator by yielding each result of the previous expression and returning the generator object itself.

In simpler terms:

- The outer loop iterates over the `books`.
- For each book, it filters out any books that don't have an "author" key.
- The inner loop yields (or returns) the author's name for each remaining book.
- Finally, `yield from ...` turns this into a generator that can be used in loops or other parts of your code.

This is equivalent to writing:

```python
def process_books(books):
    for book in books:
        if "author" in book:
            yield book["author"]
```

But the first version is more concise and doesn't use recursion (since it's using `yield from`), which makes it slightly more efficient.

This kind of code can be useful when you want to iterate over a list or other iterable, but only include certain items. Without `yield from`, you'd have to write something like this:

```python
def process_books(books):
    for book in books:
        if "author" in book:
            yield book["author"]
```

But with `yield from` you can do it more succinctly:

```python
def process_books(books):
    return {book.get("author") for book in books if "author" in book}
```